# Finetuning a GPT-2 model using data scraped from NYUAD confessions and NYUAD crushes and compliments


 Train a GPT-2 Text-Generating Model w/ GPU For Free 

by [Max Woolf](http://minimaxir.com) with modifications by [Michael Ang](https://michaelang.com) for [A.rt I.ntel NYUAD IM](https://github.com/nyuad-im/artintel)

*Last updated: November 10th, 2019*

*Updated by Mangtronix: October 7, 2020*

Retrain an advanced text generating neural network on any text dataset **for free on a GPU using Collaboratory** using `gpt-2-simple`!

For more about `gpt-2-simple`, you can visit [this GitHub repository](https://github.com/minimaxir/gpt-2-simple). You can also read my [blog post](https://minimaxir.com/2019/09/howto-gpt2/) for more information how to use this notebook!


To get started:

1. Copy this notebook to your Google Drive to keep it and save your changes. (File -> Save a Copy in Drive)
2. Make sure you're running the notebook in Google Chrome.
3. Run the cells below:


In [ ]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple


In [ ]:
import gpt_2_simple as gpt2
import tensorflow as tf
from datetime import datetime
from google.colab import files

## GPU

Colaboratory uses either a Nvidia T4 GPU or an Nvidia K80 GPU. The T4 is slightly faster than the old K80 for training GPT-2, and has more memory allowing you to train the larger GPT-2 models and generate more text.

You can verify which GPU is active by running the cell below.

In [ ]:
!nvidia-smi

## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [ ]:
gpt2.download_gpt2(model_name="355M")

## Uploading a Text File to be Trained to Colaboratory

Run the next cell to upload a text file to use for finetuning.

Upload **any smaller text file**  (<10 MB)

In [5]:
run_name = "gpt_uploaded"

from google.colab import files

uploaded = files.upload()

file_name = list(uploaded.keys())[0]
print("Received file " + file_name)


Saving confessions.txt to confessions.txt
Received file confessions.txt


## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [6]:
tf.reset_default_graph() # Reseting allows us to finetune multiple times
                         # Run this cell again for more finetuning

sess = gpt2.start_tf_sess()

steps = 100 # Number of times to train
sample_every = 10 # Ask the model to generate "original" text as we train so we can
                  # see the progress - the generated text should start to match
                  # our sample text.
sample_length = 100 # How long of sample text to generate

gpt2.finetune(sess,
              dataset=file_name,
              model_name='355M',
              checkpoint_dir = 'checkpoint',
              steps=steps,
              run_name=run_name,
              print_every=1,
              sample_every=sample_every, 
              sample_length=sample_length,
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.
Loading checkpoint models/355M/model.ckpt
INFO:tensorflow:Restoring parameters from models/355M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:00<00:00,  2.26it/s]


dataset has 65600 tokens
Training...
[1 | 9.59] loss=3.29 avg=3.29
[2 | 10.47] loss=3.20 avg=3.24
[3 | 11.35] loss=3.18 avg=3.22
[4 | 12.23] loss=3.33 avg=3.25
[5 | 13.11] loss=3.31 avg=3.26
[6 | 14.00] loss=3.01 avg=3.22
[7 | 14.88] loss=2.81 avg=3.16
[8 | 15.76] loss=3.28 avg=3.17
[9 | 16.64] loss=3.36 avg=3.20
[10 | 17.52] loss=3.13 avg=3.19
======== SAMPLE 1 ========
 a few questions; "
" Will the company continue to accept donations of any size after this event? "
" If the company has to close their site, will they take it down after? "
" I believe the fact that most of this is on the internet, has changed the way that I feel about people working and living here. It's hard to just work through it alone and feel that anyone cares. "
" I've been coming to this university for the last two years,

[11 | 22.07] loss=2.64 avg=3.14
[12 | 22.95] loss=2.59 avg=3.09
[13 | 23.84] loss=2.85 avg=3.07
[14 | 24.72] loss=3.38 avg=3.09
[15 | 25.60] loss=3.33 avg=3.11
[16 | 26.48] loss=2.09 avg=3.0

After the model is trained, you can copy the checkpoint folder to your own Google Drive (see cells further below).

## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [7]:
gpt2.generate(sess, run_name=run_name)

bdync is a library to mirror your favorite apps
# 3840 " dbus-daemon: don't forget to disable the warning dialogs on your home page
# 3839 " any suggestions for improvement?
# 3838 " I feel like I've been doing the wrong thing since I was a kid, like I'm normal but not normal enough "
# 3837 " I don't know why I have to be so mean about people who don’t understand me. Why should I have to be nice to people who don’t know me? I'm just a normal human being. "
# 3836 " I love my roommate so much. He is an amazing person, and I’m so very lucky to have known him. He is a great guy, and I wish I could have known him sooner. Anyways, I just wanted to say how much I appreciate your presence. I really do. Also, I love your smile. "
# 3835 " Is it ok to post selfies in the comment section? Like I said, I don't know anything about this and I'm just trying to help out people who need it. "
# 3834 " "I don't know if I should've just said that I'm a girl who is into boys. I know a lot of guys think 

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [8]:
gpt2.generate(sess,
              run_name=run_name,
              length=250,
              temperature=0.7,
              prefix="Put your starting text here for the model to continue",
              nsamples=5,
              batch_size=5
              )

Put your starting text here for the model to continue their learning process. When you are done, leave a comment. I will do my best to help. "
# 3939 " I just wanted to give you a huge shoutout for the most amazing month of your life. You are truly amazing "
# 3938 " I have a twin sister who I love so much and I can't imagine living without her. But ever since I met this guy from NYUAD, I can't imagine living without her. I can't imagine living without her. I can't imagine living without her. I can't imagine living without her. I can't imagine living without her. I can't imagine living without her. I can't imagine living without her. I can't imagine living without her. I can't imagine living without her. I can't imagine living without her. I can't imagine living without her. I can't imagine living without her. I can't imagine living without her. "
# 3937 " I just wanted to say I'm sorry if I offended you with the caption for this chapter. I know it was a little harsh. I was just trying

For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [9]:
gen_file = 'gpt2_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      run_name=run_name,
                      destination_path=gen_file,
                      length=500,
                      prefix="Put your starting text here",
                      temperature=0.7,
                      nsamples=10,
                      )

In [10]:
# Copy to Drive if mounted
import os
def is_mounted():
  return os.path.isdir('/content/drive')

if is_mounted():
  print("Copying %s to Drive" % gen_file)
  gpt2.copy_file_to_gdrive(gen_file)

In [11]:
# Download the file may have to run twice to get file to download
files.download(gen_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [12]:
gpt2.mount_gdrive()

Mounted at /content/drive


Copy the model trained in this session to your Google Drive

In [13]:
print("Copying run " + run_name + " to Google Drive")
gpt2.copy_checkpoint_to_gdrive(run_name=run_name)
print("Finished copying")

Copying run gpt_uploaded to Google Drive
Finished copying


## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [14]:
gpt2.copy_checkpoint_from_gdrive(run_name=run_name)

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

In [15]:
tf.reset_default_graph()

sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name=run_name)

Loading checkpoint checkpoint/gpt_uploaded/model-100
INFO:tensorflow:Restoring parameters from checkpoint/gpt_uploaded/model-100


Now we can generate text from the loaded model. You can also go back and do more finetuning, then save the model again.

In [16]:
gpt2.generate(sess, run_name=run_name, length=250)

# 3839 " I can't believe I just got my first "Real Housewives of Orange County" paycheck. This is so unreal I'm speechless. "
# 3838 " The next time you're going to the doctor, just remember to take care of yourself first. "
# 3837 " "What's the single most frustrating thing you've ever done in your entire life?" "
# 3836 " My boyfriend has been harassing me for months about my weight. I literally told him I was not interested. I thought it was over. But then I remembered that he's been going through a breakup, and that's when I started feeling really insecure. He's been telling me he's single too, and that his ex-girlfriend cheated on him. I guess he's just not that into me. But now I see he still wants to be with me. My head is spinning. How can I talk to him about this? "
# 3835 " "I am really ill at ease with myself. I might be sick or in my mid-20's, but I truly am myself. Even if I don't look or act the way I want to, I know that I am. I am who I


## Generate Text From The Pretrained Model

If you want to generate text from the pretrained model, not a finetuned model, pass `model_name` to `gpt2.load_gpt2()` and `gpt2.generate()`.

This is currently the only way to generate text from the 774M or 1558M models with this notebook.

In [17]:
#model_name = "774M"
model_name = "1558M"


gpt2.download_gpt2(model_name=model_name)

Fetching checkpoint: 1.05Mit [00:00, 507Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 72.2Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 559Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001:  76%|████████████▏   | 4.72G/6.23G [01:31<00:29, 51.4Mit/s]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-17-10dcd3a2b1bf>", line 5, in <module>
    gpt2.download_gpt2(model_name=model_name)
  File "/usr/local/lib/python3.6/dist-packages/gpt_2_simple/gpt_2.py", line 93, in download_gpt2
    file_name=file_name)
  File "/usr/local/lib/python3.6/dist-packages/gpt_2_simple/gpt_2.py", line 60, in download_file_with_progress
    f.write(chunk)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 1823, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent ca

KeyboardInterrupt: ignored

In [ ]:
sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess, model_name=model_name)

In [ ]:
gpt2.generate(sess,
              model_name=model_name,
              prefix="SOLTAN: Why are you here?\nMASKED MAN: Didn't you ask me to come?\nSOLTAN: Most certainly I did not!\n",
              length=1000,
              temperature=0.7,
              #top_p=0.9,
              nsamples=1,
              batch_size=1
              )

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [ ]:
!kill -9 -1

# LICENSE

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.